#### XGBoost


In [ ]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('boston.csv')
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [7]:
df.keys()

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [13]:
df.rename(columns={'MEDV':'PRICE'},inplace=True)


In [14]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [15]:
x = df.drop('PRICE',axis = 1)
y = df['PRICE']

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2, random_state=123)

In [18]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.3,learning_rate=0.1,max_depth=5,alpha = 10, n_estimators=10)
xgb_model.fit(x_train,y_train)

C:\Users\Vivek\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [07:23:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  bst.update(dtrain, iteration=i, fobj=obj)


XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=10,
             n_jobs=None, ...)

In [19]:
y_pred = xgb_model.predict(x_test)

In [21]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print('RMSE: %f' % (rmse))

RMSE: 5.796800


In [22]:
data_matrix = xgb.DMatrix(data=x,label=y)
data_matrix

In [ ]:
params = {
    'objective': 'reg:linear','colsample_bytree':0.3,
    'learning_rate':0.1,'max_depth':5,'alpha':10}

cv_results = xgb.cv(dtrain = data_matrix, params = params, 
                    nfold = 3, num_boost_round=50,early_stopping_rounds=10,metrics='rmse',
                    as_pandas=True,seed=123)

C:\Users\Vivek\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:209: UserWarning: [07:55:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  return getattr(self.bst, name)(*args, **kwargs)
C:\Users\Vivek\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:215: UserWarning: [07:55:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.bst.update(self.dtrain, iteration, fobj)


In [26]:
print((cv_results['test-rmse-mean']).tail(1))

49    3.788392
Name: test-rmse-mean, dtype: float64


In [27]:
cv_results

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,8.770798,0.177981,8.863473,0.362677
1,8.417938,0.164167,8.572054,0.376347
2,8.000628,0.143783,8.253671,0.354402
3,7.446014,0.109524,7.761689,0.324164
4,6.944920,0.085525,7.343192,0.304991
5,6.614792,0.063532,7.107513,0.304286
6,6.185815,0.045513,6.752788,0.271452
7,5.817057,0.032837,6.442863,0.236545
8,5.621076,0.047843,6.312473,0.235734
9,5.283924,0.062616,6.028155,0.224599
